# CMP 5006 - Information Security 


## Asymmetric Encryption


### Alejandro Proano, PhD. 

## What is Asymmetric Encryption?
- Also known as Public Key Cryptography
- Uses two different keys:
  - Public Key (shared openly)
  - Private Key (kept secret)
- Keys are mathematically related but computationally difficult to derive


## Key Characteristics
- Encryption Key ≠ Decryption Key
- Public key can encrypt
- Only private key can decrypt
- Enables secure communication without shared secret


## RSA Algorithm

- Rivest-Shamir-Adleman algorithm.
- It's one of the most common encryption methods in the world. 
- Steps:
    - Key generation
    - Encryption
    - Decryption

## Key Generation:

- Select two large prime numbers $p$ and $q$.
- Compute  $n = p \times q$. The value $n$ is used as the modulus for both the public and private keys.
- Calculate the totient function $\phi(n) = (p-1) \times (q-1)$.
- Choose an integer $e$ such that $1 < e < \phi(n)$ and $e$ is coprime with $\phi(n)$. 
    - $e$ becomes the public exponent.
- Determine $d$ as the modular multiplicative inverse of $e$ modulo $\phi(n)$. 
    - $d \times e \equiv 1 \ (\text{mod} \ \phi(n))$. 
    - $d$ is the private exponent.

## Public and Private Keys:

- Public Key: $(e, n)$
- Private Key: $(d, n)$


## Encryption:
- To encrypt a message $M$, convert $M$ into an integer $m$ such that $0 \leq m < n$.
- Compute the ciphertext $c$ using the public key: $c = m^e \ (\text{mod} \ n)$.


## Decryption:
- To decrypt the ciphertext $c$, use the private key: $m = c^d \ (\text{mod} \ n)$.
- Convert the integer $m$ back to the original message $M$.


## Example:

### Key Generation
- Let p = 11, q = 13
- n = p * q = 143

### Calculate Phi (Totient)
- φ(n) = (p-1) * (q-1)
- φ(n) = 10 * 12
- φ(n) = 120


### Choose Public Exponent (e)
- Requirements:
  - 1 < e < φ(n)
  - e must be coprime with φ(n)
- Let e = 7
- Verify: gcd(7, 120) = 1


### Calculate Private Exponent (d)
- Find d such that: (d * e) ≡ 1 (mod φ(n))
- Use modular multiplicative inverse
- d = 103 (can be calculated using extended Euclidean algorithm)
- Verification: (7 * 103) mod 120 = 1



### Public and Private Keys
- Public Key: (n, e) = (143, 7)
- Private Key: (n, d) = (143, 103)


### Encryption Process
- Choose a message: M = 42
- Encryption: C = M^e mod n
- C = 42^7 mod 143
- C = 42 * 42^6 mod 143
- C = 110


### Decryption Process
- Decrypt Ciphertext: M = C^d mod n
- M = 110^103 mod 143
- M = 42


In [2]:
### Python Implementation

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def mod_inverse(e, phi):
    def egcd(a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = egcd(b % a, a)
            return (g, x - (b // a) * y, y)
    
    g, x, _ = egcd(e, phi)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    else:
        return x % phi

def generate_keypair(p, q):
    # Calculate n and phi
    n = p * q
    phi = (p-1) * (q-1)
    
    # Choose e
    e = 7  # In practice, this would be more carefully selected
    
    # Calculate d
    d = mod_inverse(e, phi)
    
    return ((n, e), (n, d))

def encrypt(pk, plaintext):
    n, e = pk
    cipher = pow(plaintext, e, n)
    return cipher

def decrypt(pk, ciphertext):
    n, d = pk
    plain = pow(ciphertext, d, n)
    return plain


In [3]:
# Example usage
p, q = 11, 13
public_key, private_key = generate_keypair(p, q)

message = 42
print("Original Message:", message)

ciphertext = encrypt(public_key, message)
print("Encrypted Message:", ciphertext)

decrypted_message = decrypt(private_key, ciphertext)
print("Decrypted Message:", decrypted_message)


Original Message: 42
Encrypted Message: 81
Decrypted Message: 42


## Limitations of This Simple Example
- Uses small prime numbers
- Demonstrates concept, not production-ready
- Real-world implementations require:
  - Larger primes
  - Padding schemes
  - Additional security measures


## Key Security Insights
- Larger primes provide stronger encryption
- Key length typically 2048+ bits in practice
- Never reveal private key
- Regularly rotate keys


## Practical Considerations
- Computational complexity increases with key size
- Prime number selection critical
- Quantum computing potential threat


# RSA Vulnerabilities and Disadvantages


## Prime Number Generation Weaknesses

Vulnerability Example:
- Poorly generated prime numbers
- Predictable p and q values
- Increased risk of factorization

Real-World Scenario:
Debian OpenSSL bug (2008)
- Predictable random number generation
- Reduced entropy in key creation
- Millions of vulnerable cryptographic keys
- [Interesting article](https://www.schneier.com/blog/archives/2008/05/random_number_b.html)



## Computational Attacks

Threat Vectors:
- Number Field Sieve algorithm
- Quantum computing potential
- Decreasing computational complexity

Practical Example:
- 512-bit RSA keys cracked in 2009
- 768-bit key factored in 2010
- Demonstrating increasing factorization capabilities
- [Interesting article](https://en.wikipedia.org/wiki/Texas_Instruments_signing_key_controversy)



## Side-Channel Attacks

- Attacks that extract secret information by analyzing physical implementation details
- Exploit information leakage through:
  - Execution time
  - Power consumption
  - Electromagnetic emissions
  - Sound
  - Other observable system characteristics


## Timing attack

- Different computational paths take different amounts of time
- Execution time varies based on:
  - Input data
  - Secret key values
  - Internal algorithm operations
  
- In RSA:
    - the attacker needs to have the target system decrypt several carefully selected values of C
    - by analyzing the timing variation, the attacker can obtain the key 1 bit at the time

Source: P.Kocher, "Timing attacks on implementations of Diffie-Hellman, RSA, DSS, and other systems"

In [ ]:

def decrypt(ciphertext, private_key):
    start_time = time.time()
    plaintext = pow(ciphertext, private_key, n)
    end_time = time.time()
    
    # Execution time varies based on key bits
    execution_time = end_time - start_time
    return plaintext, execution_time

# Attacker can infer key details from timing variations



## Attack Limitations
- Requires extremely precise time measurement
- Needs multiple execution attempts
- Affected by:
  - System noise
  - Environmental variations
  - Hardware inconsistencies


## Defensive Techniques

- Constant-Time Implementations
- Add random execution delays
- Implement cryptographic blinding
- Use noise injection techniques


## Power Analysis Attack

Attack Methodology:
- Measure power consumption during decryption
- Correlate power signatures with key bits
- Reconstruct secret key information

Vulnerable Scenarios:
- Hardware implementations
- Smart cards

